In [ ]:
from scripts.scale_transform_magnitude import compute_stm

import mirdata
import librosa
from tqdm import tqdm
import pandas as pd
from pathlib import Path


%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
groove_dataset = mirdata.initialize("groove_midi")
# groove_dataset.download()
groove_dataset.validate()

In [26]:
# groove_dataset._metadatafill
metadata = pd.read_csv("~/mir_datasets/groove_midi/info.csv")
metadata_beats = metadata[metadata["beat_type"] == "beat"]  # select only beats

metadata_beats = metadata_beats[metadata_beats["duration"] >= 30]
metadata_beats = metadata_beats[metadata_beats["time_signature"] >= "4-4"]

print("duration summary in seconds: \n", metadata_beats["duration"].describe())

metadata_beats["genre"] = metadata_beats["style"].apply(
    lambda x: x.split("/")[0]
)  # create genre column based on style

metadata_beats = metadata_beats[["genre", "style", "audio_filename", "duration"]] # get rid of unnecessary columns
metadata_beats = metadata_beats.reset_index(drop=True) # resetting index is neede for interactive plot

print(metadata_beats.genre.unique())

# Counting occurrences of each genre
genre_counts = metadata_beats['genre'].value_counts()

# Identifying the top n most frequent genres
top_n_genres = genre_counts.head(6).index

metadata_beats = metadata_beats[metadata_beats['genre'].isin(top_n_genres)]

metadata_beats.genre.value_counts()

duration summary in seconds: 
 count     340.000000
mean      130.695639
std       118.145679
min        30.060481
25%        53.633509
50%       112.187331
75%       172.502978
max      1394.173889
Name: duration, dtype: float64
['soul' 'funk' 'hiphop' 'rock' 'neworleans' 'dance' 'latin' 'afrocuban'
 'reggae' 'jazz' 'country' 'gospel' 'punk' 'afrobeat' 'blues'
 'middleeastern' 'pop' 'highlife']


genre
rock      107
latin      47
funk       44
jazz       34
soul       25
hiphop     22
Name: count, dtype: int64

In [ ]:
# preparing the data and computing stm
groove_midi_path = Path("~/mir_datasets/groove_midi/").expanduser()
features = []
for row in tqdm(metadata_beats.itertuples(index=False), total=metadata_beats.shape[0]): # TODO: find a more efficient way to loop
    try:
        if row.audio_filename == None: continue
        # TODO: segment audio file?
        y, sr = librosa.load(groove_midi_path / row.audio_filename, sr=None, duration=30)
        features.append(
            compute_stm(y=y, sr=sr)
        ) 
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
# f = []
# l = []
# for _, track, in tqdm(groove_dataset.load_tracks().items()):
#     if track.audio_path != None and track.beat_type == "beat" and duration:
#         try:
#             y, sr = librosa.load(track.audio_path, sr=None) 
#             stm = compute_stm(y, sr, with_padding=True, autocor_window_type="hamming")[:200]
#             f.append(stm)
#             l.append(track.style)
#         except Exception as e:
#             print(f"error: {e}")